<a href="https://www.kaggle.com/code/yassmen/data-science-project?scriptVersionId=176319598" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Basic information about dataset

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import plotly.express as px # create interactive charts

In [ ]:
df = pd.read_csv("/kaggle/input/e-commerce-customer-for-behavior-analysis/ecommerce_customer_data_large.csv")
df.info() 
df.shape
df.head() 

In [ ]:
df.nunique()

# 2. Pre-Processing data

In [ ]:
# check duplicates, returns all duplicated rows 
df[df.duplicated(keep=False)] 

*Customer Age* and *Age* columns seem to contain identical data, one should be removed.

In [ ]:
df=df.drop('Age',axis=1)

In [ ]:
df.head() 

In [ ]:
# checking for missing values
df.isnull().sum()

In [ ]:
# Replace all NaN to 0 for Returns
# Convert dtype to integer
df['Returns'] = df['Returns'].fillna(0).astype(int)

#check non-null count & dtype
df['Returns'].info()

In [ ]:
# Convert `Purchase Date` dtype to Datetime
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'], format='%Y-%m-%d %H:%M:%S')

# Split Year and Month into new columns
df['Purchase Year'] = df['Purchase Date'].dt.year
df['Purchase Month'] = df['Purchase Date'].dt.month_name()

df[['Purchase Date','Purchase Year','Purchase Month']]

In [ ]:
# Re-calculate total purchase amount
# Remove incorrect `Total Purchased Amount` column
df['Total Price'] = df['Product Price'] * df['Quantity']
df = df.drop(['Total Purchase Amount'], axis=1)

In [ ]:
#rename each column
cols = ['Customer ID','Customer Name','Purchase Date','Purchase Year','Purchase Month','Product Category','Product Price','Quantity','Total Price','Payment Method','Returns','Gender','Customer Age','Churn']
df = df[cols]

In [ ]:
df.info()

In [ ]:
# Save cleaned dataset to a csv file or copy to a new dataframe
Clean_df = df.copy()

# 3. Visualization

In [ ]:
Clean_df.describe(include='all')

In [ ]:
Clean_df.head(3)

# Gender analysis

In [ ]:
# Overview stats of Gender and total price spent
gender_summary = Clean_df.groupby('Gender').agg(total_count=('Customer ID','size'),
                                            min_amount=('Total Price', 'min'),
                                            max_amount=('Total Price', 'max'),
                                            median_amount=('Total Price','median'),
                                            total_amount=('Total Price','sum'),
                                            average_amount =('Total Price','mean'),
                                            total_returned=('Returns','sum'),
                                            return_rates=('Returns','mean'),
                                            total_churned=('Churn','sum'),
                                            churn_rates=('Churn','mean')).reset_index()

gender_summary


In [ ]:
## 1. Gender distribution 
# calculate count for each gender
gender_count = Clean_df['Gender'].value_counts()
# create pie chart
fig1 = px.pie(values=gender_count, names=gender_count.index,
              color = gender_count.index,
              color_discrete_map={'Female':'#FF6692', 'Male':'#3366CC'},
              title='1. Gender Distribution')
# adjust in-graph text
fig1.update_traces(textposition='inside', textinfo='text', 
                   texttemplate='%{label}<br>%{value}<br>(%{percent})')
# adjust chart size and remove legend
fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=400, height=400, showlegend=False)


# display graph
fig1.show()

In [ ]:
## 2. Purchase price distribution by gender
# Calculate the percentage of total purchased amount for each gender
sum_amount = gender_summary['total_amount'].sum()
gender_summary['total_amount_percentage'] = (gender_summary['total_amount']/sum_amount*100).round(2)

# create a bar chart 
fig2 = px.bar(gender_summary, x='total_amount',y='Gender',
              orientation='h',
              color='Gender',
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              title='2. Total Purchase Amount by Gender',
              text=gender_summary.apply(lambda x: f"${x['total_amount']:,}<br>({x['total_amount_percentage']:.2f}%)", axis=1),
              labels={'total_amount':'Total Purchased Amount ($)'})
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                  yaxis_title= None, 
                  width=550, height=350, showlegend=False)
# display graph
fig2.show()

In [ ]:
## 3. Customer churn and returns by gender
# Convert churn and return rates to percentage
gender_summary['churn_rates'] *= 100
gender_summary['return_rates'] *= 100


fig3 = px.bar(gender_summary, x='Gender', y=['total_churned', 'total_returned'],
             barmode='group', 
             title='3. Customer Churn and Returns by Gender',
             text_auto='.4s',
             labels={'value': 'Number of Customers'},
             hover_data={'churn_rates': True, 'return_rates': True}) 

# Adjust data shown in the hover text
fig3.update_traces(hovertemplate="Gender: %{x}<br>"
                                 "Total: %{y:,.0f}<br>"
                                 "Churn Rate: %{customdata[0]}%<br>"
                                 "Return Rate: %{customdata[1]}%<extra></extra>")

# Add annotations for churn/return rates
for index, row in gender_summary.iterrows():
    fig3.add_annotation(x=row['Gender'], y=row['total_churned'],
                        text=f"{row['churn_rates']:.2f}%", 
                        showarrow=False, xanchor='right', yanchor='bottom', 
                        font=dict(color='blue', size=12))

for index, row in gender_summary.iterrows():
    fig3.add_annotation(x=row['Gender'], y=row['total_returned'],
                        text=f"{row['return_rates']:.2f}%",
                        showarrow=False, xanchor='left', yanchor='bottom', 
                        font=dict(color='red', size=12))
    

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=600, height=500, 
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))

# display graph
fig3.show()

In [ ]:
## 4 Average price spent breakdown by year
# Calculate the overall average spending and prepare data for plotting
overall_avg_price = Clean_df['Total Price'].mean()
avg_price_gender_year = Clean_df.groupby(['Gender', 'Purchase Year'], as_index=False)['Total Price'].mean()

# Create box plot visualizing total price by gender and year, including overall average line
fig4 = px.box(Clean_df, x='Gender', y='Total Price', color='Purchase Year',
              title='4. Total Price Spent by Gender, Breakdown by Year',
              labels={'Total Price': 'Total Purchased Amount ($)'})

fig4.add_hline(y=overall_avg_price, line_dash="dash", line_color="#325A9B",
               annotation_text=f'Overall Average: ${overall_avg_price:.2f}',
               annotation_position="top",  # Adjusted for better visibility
               annotation_font_color="#325A9B")

fig4.update_layout(title={'x': 0.5, 'y': 0.9}, 
                   width=700, height=450,
                   xaxis_title=None)
# display graph
fig4.show()

# Age analysis

In [ ]:
# Define bins for age groups
age_bins = [0,9,19,29,39,49,59,69,np.inf]
age_labels = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70+']

# Add a new column and assign age group category to each transaction in Clean_df
Clean_df['Age Group'] = pd.cut(Clean_df['Customer Age'], bins=age_bins, labels=age_labels) 

# Check if `Age Group` is assigned correctly and added to the dataset
Clean_df[['Customer ID','Customer Age','Age Group']].sort_values('Customer Age', ascending=False)



In [ ]:
## 1. Age distribution by age
Clean_df['Age Group'] = pd.Categorical(Clean_df['Age Group'], categories=age_labels, ordered=True)

# Count each age group
age_group_count = Clean_df['Age Group'].value_counts().sort_index().reset_index(name='Count')

# Calculate the percentage for each age group
age_group_count['Percentage'] = round(age_group_count['Count'] / age_group_count['Count'].sum()*100, 2)
age_group_count['Percentage Text'] = age_group_count['Percentage'].astype(str) + '%'

# Plot a line chart with bars
fig1 = px.line(age_group_count,x='Age Group', y='Count',
               title='1. Age Distribution',
               markers=True,
               text='Percentage Text',
               labels={'Count': 'Number of Customers'})

fig1.update_traces(textposition="top center") #adjust text position for percentages 
fig1.add_bar(x=age_group_count['Age Group'], y=age_group_count['Count'], text=age_group_count['Count'], textposition='inside', name='Count')
fig1.update_traces(textfont_color='white', selector=dict(type='bar'))
fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                   xaxis_title= None,
                   yaxis_title= 'Count',
                   width=750, height=450, 
                   showlegend=False)
# display graph
fig1.show()

In [ ]:
## 2. Age distrbution by gender
age_group_gender_count = Clean_df[['Gender','Age Group']].value_counts().reset_index(name='Count')

fig2 = px.histogram(age_group_gender_count, x='Age Group', y='Count', 
              color='Gender', barmode='group', 
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              category_orders={'Age Group': age_labels},
              title='2. Age Distribution by Gender',
              text_auto=True)
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=750, height=450,
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                               y=1.02, xanchor="center",x=0.5))
# display graph
fig2.show()

In [ ]:
## 3. Average price spent by age group
overall_avg_price = Clean_df['Total Price'].mean()
avg_price_age_group = Clean_df.groupby(['Age Group'])['Total Price'].mean().reset_index(name='Avg Price')

fig3 = px.bar(avg_price_age_group, x= 'Age Group', y= 'Avg Price', 
              color = 'Avg Price',
              color_continuous_scale= "peach", 
              category_orders={'Age Group': age_labels},
              title='3. Average Price Spent by Age Group',
              text= [f"${value:.2f}" for value in avg_price_age_group['Avg Price']],
              hover_data={'Age Group':True, 'Avg Price':"${:.2f}"})

# Adjust data and format shown in the hover data
fig3.update_traces(hovertemplate="Age Group: %{x}<br>"
                                 "Average Spending: %{y:$,.2f}")

# Add a line to show the overall average for spending 
fig3.add_hline(y=overall_avg_price, line_dash="dash", line_color="#1CBE4F",
               annotation_text=f'Overall Average: ${overall_avg_price:.2f}',
               annotation_font_color="#1CBE4F",
               annotation_position="top left")

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=700, height=450,
                   xaxis_title= None, 
                   yaxis_title= 'Average Spending ($)',
                   yaxis=dict(range=[500, 800],  # Set range from 400 to 800
                              tick0=500,  # Set starting tick at 400
                              dtick=50),  # Set interval between ticks
                   showlegend=False)
# display graph
fig3.show()

In [ ]:
## 4. Customer churn and returns by age group
age_group_churned_returned = Clean_df.groupby('Age Group').agg(total_churned=('Churn','sum'),
                                                           churn_rates=('Churn','mean'),
                                                           total_returned=('Returns','sum'),
                                                           return_rates=('Returns','mean')).fillna(0).reset_index()

# Calculate the total churned/returned count and churn/return rates 
age_group_churned_returned['churn_rates'] *= 100
age_group_churned_returned['return_rates'] *= 100


fig4 = px.bar(age_group_churned_returned, x='Age Group', y=['total_churned', 'total_returned'],
             barmode='group', 
             category_orders={'Age Group': age_labels},
             title='4. Customer Churn and Returns by Age Group',
             text_auto= '.3s' ,
             labels={'value': 'Number of Customers'},
             hover_data={'churn_rates': True, 'return_rates': True}) 

# Adjust data shown in the hover text
fig4.update_traces(hovertemplate="Age Group: %{x}<br>"
                                 "Total: %{y:,.0f}<br>"
                                 "Churn Rate: %{customdata[0]}%<br>"
                                 "Return Rate: %{customdata[1]}%<extra></extra>")

fig4.update_traces(textfont={'size': 11}, textposition='inside')

# Add annotations for churn rates
for index, row in age_group_churned_returned.iterrows():
    fig4.add_annotation(x=row['Age Group'], y=row['total_churned'],
                        text=f"{row['churn_rates']:.1f}%", 
                        showarrow=False, xanchor='right', yanchor='bottom', 
                        font=dict(color='blue', size=11))

# Add annotations for return rates
for index, row in age_group_churned_returned.iterrows():
    fig4.add_annotation(x=row['Age Group'], y=row['total_returned'],
                        text=f"{row['return_rates']:.1f}%",
                        showarrow=False, xanchor='left', yanchor='bottom', 
                        font=dict(color='red', size=11))

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=800, height=500, bargap=0.1, 
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))


# display graph
fig4.show()

# Products & Sales analysis

In [ ]:
Clean_df['Return Quantity'] = Clean_df.apply(lambda row: row['Quantity'] if row['Returns'] == 1 else 0, axis=1)

product_summary_table = Clean_df.groupby('Product Category').agg(Count=('Product Category', 'size'),
                                               Order_Quantity=('Quantity', 'sum'),
                                               Total_Revenue=('Total Price','sum'),
                                               Average_Revenue=('Total Price','mean'),
                                               Return_Count=('Return Quantity', 'sum')).reset_index()

# Calculate the overall percentage for product quantity sold
total_order_quantity = product_summary_table['Order_Quantity'].sum()
product_summary_table['Overall_Percentage'] = (product_summary_table['Order_Quantity'] / total_order_quantity) * 100

# Calculate the percentage for return
product_summary_table['Return_Rate'] = round((product_summary_table['Return_Count'] / product_summary_table['Order_Quantity']) * 100, 2)

product_summary_table

In [ ]:
# 1. Product Distribution and Returns by category
# Set the category order for plotting colors
category_order = ["Electronics", "Clothing", "Home", "Books"]

fig1 = px.bar(product_summary_table, 
             x='Product Category', 
             y=['Order_Quantity', 'Return_Count'],
             barmode='overlay', opacity=0.7,
             color='Product Category',
             color_discrete_sequence=px.colors.qualitative.T10,
             pattern_shape = 'variable',
             category_orders={'Product Category': category_order},
             title='Product Category Distribution vs. Returns',
             labels={'value': 'Return & Total Order Count'})

# Add overall percentage and returns rate as text on the graph
for i, row in product_summary_table.iterrows():
    fig1.add_annotation(x=row['Product Category'], y=row['Order_Quantity']-15000, text=f"Total:{row['Order_Quantity']/ 1000:,.2f}K<br>({row['Overall_Percentage']:.2f}%)",
                       font=dict(color='black', size=12), showarrow=False)

for i, row in product_summary_table.iterrows():
    fig1.add_annotation(x=row['Product Category'], y=row['Return_Count']-15000, text=f"Returned:{row['Return_Count']/ 1000:,.2f}K<br>({row['Return_Rate']:.2f}%)",
                       font=dict(color='white', size=12), showarrow=False)
    

fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=700, height=500,
                  showlegend=False)
# display graph
fig1.show()

In [ ]:
# 2. Sales Revenue by Product Category
total_revenue = Clean_df.groupby(['Product Category','Purchase Year'])['Total Price'].sum().reset_index()
total_revenue['Purchase Year'] = total_revenue['Purchase Year'].astype(str)

fig2 = px.bar(total_revenue, x='Product Category', y='Total Price',
              color='Purchase Year', barmode= 'group',
              color_discrete_sequence=px.colors.qualitative.Set2,
              category_orders={'Product Category': category_order},
              title = '2. Sales Revenue by Product Category and Year',
              labels={'Total Price': 'Total Price ($)'})
fig2.update_traces(texttemplate='%{y:$.4s}', textposition='inside', textangle=90)
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=700, height=500)
# display graph
fig2.show()

In [ ]:
# 3. Product revenue by Gender
product_revenue_gender = Clean_df.groupby(['Product Category','Gender'])['Total Price'].sum().reset_index(name='Total Revenue')

fig3 = px.bar(product_revenue_gender,x='Product Category', y='Total Revenue', 
              color='Gender',
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              barmode='group', 
              title='3. Sales Revenue by Product Category & Gender',
              category_orders={'Product Category': category_order},
              labels={'Total Revenue': 'Total Revenue ($)'})

fig3.update_traces(texttemplate='%{y:$.4s}', textposition='inside', textangle=90)

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=700, height=500,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))

# display graph
fig3.show()

In [ ]:
##4. Product revenue by Age Group
product_revenue_age = Clean_df.groupby(['Product Category','Age Group'])['Total Price'].sum().reset_index(name='Total Revenue')

fig4 = px.bar(product_revenue_age[product_revenue_age['Total Revenue'] > 0], #filter out empty bars
              x='Age Group', y='Total Revenue',
              color='Product Category', barmode= 'group',
              color_discrete_sequence=px.colors.qualitative.T10,
              category_orders={'Product Category': category_order},
              title = '4. Product Sales Revenue by Category & Age Group',
              labels={'Total Revenue': 'Total Revenue ($)'})

fig4.update_traces(texttemplate='%{y:$.3s}', textangle=90)

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=800, height=500,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))


# display graph
fig4.show()

# Time Series Pattern analysis

In [ ]:
# overview of yearly sales revenue
annual_sales_summary = Clean_df.groupby('Purchase Year')['Total Price'].agg(['min', 'max','mean','median','sum',])
annual_sales_summary

In [ ]:
# Set month names in order and dtype to category
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

Clean_df['Purchase Month'] = pd.Categorical(Clean_df['Purchase Month'], 
                                        categories=month_order, 
                                        ordered=True)

# Calculate the total and average of monthly sales revenue
monthly_sales = Clean_df.groupby(['Purchase Year', 'Purchase Month']).agg(Count=('Customer ID', 'size'),
                                                                      Total_Revenue=('Total Price', 'sum'),
                                                                      Avg_Revenue=('Total Price', 'mean')).reset_index()

In [ ]:
## 1. Monthly count of orders, split by years
# Convert years to string for bar chart
monthly_sales['Purchase Year'] = monthly_sales['Purchase Year'].astype(str)

fig1 = px.bar(monthly_sales, x='Purchase Month', y='Count', 
               color='Purchase Year', facet_row='Purchase Year',
               title='1. Number of Orders per Month (by Year)', 
               labels={'Count': 'Order Count', 'Purchase Year': 'Year'},
               hover_data={'Count': ':,.0f'},
               text_auto=True)

fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None, 
                  width=650, height=600,
                  showlegend=False)
# display graph
fig1.show()

In [ ]:
## 2. Compare total monthly sales
# Filter out data points where 'Total_Revenue' is 0
fig2 = px.area(monthly_sales[monthly_sales['Total_Revenue'] != 0], x='Purchase Month', y='Total_Revenue', 
              color='Purchase Year', facet_col='Purchase Year',
              color_discrete_sequence=px.colors.qualitative.Dark2,
              title='2. Total Monthly Sales Revenue (by Year)',
              labels={'Total_Revenue': 'Total Revenue ($)', 'Purchase Year': 'Year'},
              hover_data={'Total_Revenue': ':$,.0f'},
              markers=True)

fig2.update_yaxes(tickprefix="$")

fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  xaxis2_title= None,
                  xaxis3_title= None,
                  xaxis4_title= None,
                  width=800, height=350,
                  showlegend=False)
# display graph
fig2.show()

In [ ]:
## 3. Compare average monthly sales
fig3 = px.line(monthly_sales, x='Purchase Month', y='Avg_Revenue', 
              color='Purchase Year', facet_col='Purchase Year',
              color_discrete_sequence=px.colors.qualitative.Set2,
              title='3. Average Monthly Sales Revenue (by Year)',
              labels={'Avg_Revenue': 'Average Revenue ($)', 'Purchase Year': 'Year'},
              hover_data={'Avg_Revenue': ':$,.0f'},
              markers=True)

fig3.update_yaxes(tickprefix="$")

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  xaxis2_title= None,
                  xaxis3_title= None,
                  xaxis4_title= None,
                  width=800, height=350,
                  yaxis=dict(range=[700, 840]), 
                  showlegend=False)
# display graph
fig3.show()


In [ ]:
## 4. Monthly return and churn rate by year    
df_return_churn = Clean_df[['Customer ID','Purchase Date','Returns','Churn']]
df_return_churn['Purchase Date'] = df_return_churn['Purchase Date'].dt.strftime('%Y-%m')
rates = df_return_churn.groupby('Purchase Date').agg(
    Return_Rate=('Returns', 'mean'), Churn_Rate=('Churn', 'mean')).reset_index()

# Convert rates to percentage
rates['Return_Rate'] *= 100
rates['Churn_Rate'] *= 100

fig4 = px.line(rates, x='Purchase Date', y=['Return_Rate','Churn_Rate'], 
              title='4. Monthly Return and Churn Rates Over time',
              markers=True,
              hover_data={'variable':':name'})


fig4.update_traces(hovertemplate="Type: %{customdata}<br>"
                                 "Purchase Date: %{x|%Y-%m}<br>"
                                 "Rate: %{y:.2f}%")

# Calculate the positions for annotations
max_return_rate = rates.loc[rates['Return_Rate'].idxmax()]
min_return_rate = rates.loc[rates['Return_Rate'].idxmin()]
max_churn_rate = rates.loc[rates['Churn_Rate'].idxmax()]
min_churn_rate = rates.loc[rates['Churn_Rate'].idxmin()]

annotations = [
    {'x': max_return_rate['Purchase Date'], 'y': max_return_rate['Return_Rate']+1, 
     'text': f"highest: {max_return_rate['Return_Rate']:.2f}%", 'showarrow': False},
    {'x': min_return_rate['Purchase Date'], 'y': min_return_rate['Return_Rate']-1, 
     'text': f"lowest: {min_return_rate['Return_Rate']:.2f}%", 'showarrow': False},
    {'x': max_churn_rate['Purchase Date'], 'y': max_churn_rate['Churn_Rate']+1, 
     'text': f"highest: {max_churn_rate['Churn_Rate']:.2f}%", 'showarrow': False},
    {'x': min_churn_rate['Purchase Date'], 'y': min_churn_rate['Churn_Rate']-1, 
     'text': f"lowest: {min_churn_rate['Churn_Rate']:.2f}%", 'showarrow': False} ]

for annotation in annotations:
    fig4.add_annotation(x=annotation['x'], y=annotation['y'], text=annotation['text'], 
                        showarrow=annotation['showarrow'], font=dict(size=11))

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  yaxis_title= 'Rates (%)',
                  width=750, height=450,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))   
# display graph
fig4.show()

In [ ]:
Clean_df


# 4. Clustring

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import seaborn as sns


customer_data = df.groupby('Customer Name').agg({
    'Total Purchase Amount': ['sum', 'mean'],
    'Returns': 'sum',
    'Churn': 'max'
}).reset_index()
customer_data.columns = ['Customer Name', 'Total Purchases', 'Average Purchase Amount', 'Total Returns', 'Churn Status']

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(customer_data[['Total Purchases', 'Average Purchase Amount', 'Total Returns', 'Churn Status']])

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
scaled_data_imputed = imputer.fit_transform(scaled_data)

# Apply KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=42)
customer_data['Cluster'] = kmeans.fit_predict(scaled_data_imputed)

# Analyze other metrics by cluster
cluster_summary = customer_data.groupby('Cluster').agg({
    'Total Purchases': 'mean',
    'Average Purchase Amount': 'mean',
    'Total Returns': 'mean',
    'Churn Status': 'mean'
}).reset_index()

print(cluster_summary)
import plotly.express as px

# Assuming 'customer_data' is your DataFrame with the clustering results
fig = px.scatter(customer_data, x='Total Purchases', y='Average Purchase Amount',
                 color='Cluster', title='Customer Segmentation based on Total Purchases and Average Purchase Amount',
                 labels={'Cluster': 'Cluster Group'})

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()


# 5. STANDARDIZE THE DATA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
# Drop non-numeric columns for simplicity
Clean_df = Clean_df.drop(columns=['Customer ID', 'Customer Name', 'Purchase Date','Purchase Month','Payment Method','Gender','Age Group'])

# Impute missing values
for column in Clean_df.columns:
    if Clean_df[column].dtype == 'object':  # Categorical column
        Clean_df[column].fillna(Clean_df[column].mode()[0], inplace=True)
    else:  # Numerical column
        Clean_df[column].fillna(Clean_df[column].mean(), inplace=True)
        
# Identify columns with dtype 'object' and encode them
for column in Clean_df.columns:
    if Clean_df[column].dtype == 'object':
        le = LabelEncoder()
        Clean_df[column] = le.fit_transform(Clean_df[column])

In [ ]:
# Split data into features and target
X = Clean_df.drop(columns=['Churn'])
y = Clean_df['Churn']


In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
X_df=pd.DataFrame(X,columns=['Purchase Year', 'Product Category', 'Product Price', 'Quantity','Total Price','Returns','Customer Age','Return Quantity'])

In [ ]:
Clean_df.drop('Churn',axis=1).head()

In [ ]:
X_df.head()

# 6. PCA

In [ ]:
# PCA PROJECTION TO 2D
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
principalDf.head()

In [ ]:
finalDf = pd.concat([principalDf, Clean_df[['Churn']]], axis = 1)
finalDf.head(500)

In [ ]:
fig = px.scatter(finalDf, x='principal component 1', y='principal component 2',
                 color='Churn', # This will set different colors for different churn categories
                 title='2 component PCA',
                 labels={'principal component 1': 'Principal Component 1', 'principal component 2': 'Principal Component 2'},
                 color_discrete_map={'0-completed': 'red', '1-uncompleted': 'green'}) # Maps your Churn categories to specific colors

fig.update_layout(legend_title_text='Churn')
fig.show()


In [ ]:
pca.explained_variance_ratio_

In [ ]:
# IMPORT AND APPLY PCA
from sklearn.decomposition import PCA

# Make an instance of the Model
pca = PCA(.95)

In [ ]:
pca.fit(X_train)

In [ ]:
# APPLY THE MAPPING (TRANSFORM) TO THE TRAINING SET AND THE TEST SET
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_train_pca.shape

# 6. Apply ML models

# 1- Knn Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, roc_auc_score, auc, confusion_matrix
# Build and train the knn model
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train_pca, y_train)

In [ ]:
# Predict on the test set
y_pred = knn.predict(X_test_pca)


In [ ]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
error_rate = []
# Will take some time
for i in range(1,40):

    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train_pca,y_train)
    pred_i = knn.predict(X_test_pca)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
# Assuming 'error_rate' is a list or array with your error rate data
data = {'K': range(1, 40), 'Error Rate': error_rate}
New_df = pd.DataFrame(data)

# Create the line plot with Plotly Express
fig = px.line(New_df, x='K', y='Error Rate', title='Error Rate vs. K Value',
              markers=True, line_shape='linear')
fig.update_traces(line=dict(color='blue', dash='dash'),
                  marker=dict(color='red', size=10, symbol='circle'))
fig.update_layout(xaxis_title='K', yaxis_title='Error Rate')

# Show the plot
fig.show()


# 2- Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
dtree = DecisionTreeClassifier()
dtree.fit(X_train_pca,y_train)

In [ ]:
predictions = dtree.predict(X_test_pca)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
feature_names = list(Clean_df.columns[1:])
feature_names

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 20), dpi=300)  
tree.plot_tree(dtree, feature_names=feature_names, filled=True, ax=ax)
#plt.close()
plt.show()

# 3- Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train_pca,y_train)

In [ ]:
predicted = logmodel.predict(X_test_pca)


print(confusion_matrix(y_test, predicted))

In [ ]:
print(classification_report(y_test,predicted))

# 4- Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model using the PCA-transformed data
rf_model.fit(X_train_pca, y_train)

In [ ]:
#  make predictions on the test set
y_pred = rf_model.predict(X_test_pca)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Calculate the accuracy of the model on the test set
print("Accuracy of Random Forest on PCA-reduced data:", accuracy_score(y_test, y_pred))

# Print out the classification report for more detailed metrics
print("Classification Report for Random Forest on PCA-reduced data:\n", classification_report(y_test, y_pred))


# 5- SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Initialize the SVM classifier
svm_model = SVC(kernel='linear', C=1)  # You can change the kernel and C value based on your dataset

# Train the model using the PCA-transformed data
svm_model.fit(X_train_pca, y_train)


In [ ]:
# Predict on the PCA-reduced test set
y_pred_svm = svm_model.predict(X_test_pca)

In [ ]:
# Calculate the accuracy of the model on the test set
print("Accuracy of SVM on PCA-reduced data:", accuracy_score(y_test, y_pred_svm))

# Print out the classification report for more detailed metrics
print("Classification Report for SVM on PCA-reduced data:\n", classification_report(y_test, y_pred_svm))
